<a href="https://colab.research.google.com/github/anmolsabharwal/RedDevils/blob/master/RD_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
from google.colab import drive
import datetime
import plotly.figure_factory as ff
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor


drive.mount('/content/drive')
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
gws = sqlite3.connect('/content/drive/My Drive/FPL1.db')
gws = pd.read_sql_query("SELECT * FROM gw",gws)
gws.head()

,Name,Total Points,Game Week
0,Aaron Cresswell,0,1
1,Aaron Lennon,1,1
2,Aaron Ramsey,2,1
3,Abdoulaye DoucourÈ,0,1
4,Abdul Rahman Baba,0,1


In [3]:
gws=gws.astype({'Total Points':int,'Game Week':int})

In [4]:
gws = gws.assign(id=(gws['Name']).astype('category').cat.codes)


In [5]:
gws.head(50)

,Name,Total Points,Game Week,id
0,Aaron Cresswell,0,1,0
1,Aaron Lennon,1,1,1
2,Aaron Ramsey,2,1,4
3,Abdoulaye DoucourÈ,0,1,9
4,Abdul Rahman Baba,0,1,10
5,Abel Hern·ndez,5,1,11
6,Adama Diomande,8,1,24
7,Adam Clayton,2,1,14
8,Adam Federici,0,1,15
9,Adam Forshaw,1,1,16


In [6]:
gws1=gws.copy()

In [7]:
#gws1['Total Points'].plot(kind='bar')

In [8]:
from sklearn.preprocessing import MinMaxScaler
#x = gws1[['Total Points']].values.astype(float)
scalex = MinMaxScaler()
scaley = MinMaxScaler()

#x_scaled = scale.fit_transform(x)
#y = pd.DataFrame(x_scaled)


In [9]:
gws1['Points']=y
gws1.head()


NameError: ignored

In [ ]:
gws1['Last_Week_Points'] = gws1.groupby(['id'])['Points'].shift()
gws1.head(600)

In [ ]:
gws1['Last_Week_Diff'] = gws1.groupby(['id'])['Last_Week_Points'].diff()


In [ ]:
gws1.head()

In [ ]:
gws1=gws1.dropna()
gws1.drop('Total Points',axis=1, inplace=True)


In [ ]:
gws1.head()


In [ ]:
def rmsle(real, predicted):
  sum=0.0
  for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
  return (sum/len(predicted))**0.5


In [ ]:
mean_error = []
for game_week in range(107,114):
    train = gws1[gws1['Game Week'] <  game_week]
    val = gws1[gws1['Game Week'] == game_week]

    p = val['Last_Week_Points'].values
    error = rmsle(val['Points'].values, p)
    print('Game Week %d - Error %.2f' % (game_week, error))
    mean_error.append(error)
print('Mean Error = %.2f' % np.mean(mean_error))


In [ ]:
mean_error = []
for week in range(112,114):
    train = gws1[gws1['Game Week'] < week]
    val = gws1[gws1['Game Week'] == week]

    xtr, xts = train.drop(['Points','Name'], axis=1), val.drop(['Points','Name'], axis=1)
    ytr, yts = train['Points'].values, val['Points'].values

    mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    mdl.fit(xtr, ytr)

    p = mdl.predict(xts)

    error = rmsle(yts, p)
    print('Week %d - Error %.5f' % (week, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))


In [ ]:
y_orig = pd.DatFrame(y.inverse_transform(yts))
df = pd.DataFrame({'Actual': y_orig, 'Predicted': p})
df.head()
